In [ ]:
import os
os.chdir('../')

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
plt.rcParams.update({'font.size': 24})

import clds

# Compensatory and Anticipatory Behavior

So far, it has been assumed that the rate of infection during working days would revert back to the rate observed pre-quarantine. The rate of infection may increase above pre-quarantine level if the population mixes with higher frequency during working days. This may occur initially in response to prolonged lockdown and subsequently in anticipation of the following quarantine period.

## Default Parameters

In [ ]:
suppression_start = 20
switching_start = 50

# ODE
step_size = 0.0005
n_steps = 350
    
# initial condition
N=1e7
I = 500/6
D = 20
A = 1
R = 2
T = H = E = 0
S = N - I - D - A - R - T - H - E
s0 = np.array([S, I, D, A, R, T, H, E])

# default latent parameters
alpha = 0.570
beta = 0.011
gamma = 0.456
delta = 0.011
epsilon = 0.171
theta = 0.371
zeta = 0.125
eta = 0.125
mu = 0.012
nu = 0.027
tau = 0.003
h = 0.034
rho = 0.034
kappa = 0.017
xi = 0.017
sigma = 0.017

## Open Loop

In [ ]:
# SIDARTHE with FPSP-(1, 6)

compensation = np.array([0.1*i for i in range(0, 20)])
batch_size = compensation.shape[0]# number of different values for d

lockdown_effectiveness = 0.175
steps_high = 1
steps_low = 6

model = clds.agents.BatchSIDARTHE(s0=s0, 
                    alpha='a', # variable
                    beta='b', # variable
                    gamma='g', # variable
                    delta='d', #variable
                    epsilon=epsilon,
                    theta=theta,
                    zeta=zeta,
                    eta=eta,
                    mu=mu,
                    nu=nu,
                    tau=tau,
                    h=h,
                    rho=rho,
                    kappa=kappa,
                    xi=xi,
                    sigma=sigma,
                    N=N, 
                    batch_size=batch_size,
                    step_size=step_size)

fpsp = clds.agents.BatchFPSP(beta_high=1,
                             beta_low=lockdown_effectiveness, 
                             steps_high=steps_high,
                             steps_low=steps_low, 
                             suppression_start=suppression_start, 
                             switching_start=switching_start, 
                             batch_size=batch_size)

env = clds.Composite()
env.add(model, 
        pre=lambda x: {'a': x['fpsp']*alpha,
                       'b': x['fpsp']*beta,
                       'g': x['fpsp']*gamma,
                       'd': x['fpsp']*delta},
        out='model')
env.add(fpsp, out='fpsp')

o = [env.reset()]
for i in range(n_steps):
    if i > suppression_start:
        fpsp.beta_high = (1+compensation)
    o.append(env.step()[0])
    
s = np.array([x['model'] for x in o]).swapaxes(0, 1)

fig, ax = plt.subplots(figsize=(10, 8))

indices = [0, 4, 6, 8, 10, 11]
for i in indices:
    plt.plot(np.sum(s[i,::,1:6], axis=1)/N*100, label='d={:.2f}, FPSP({},{})'.format(compensation[i], steps_high, steps_low))
ylim = plt.ylim()
plt.plot([suppression_start, suppression_start], ylim, '--', c='red')
plt.plot([switching_start, switching_start], ylim, '--', c='blue')
ax.set_xlabel('time (days)')
ax.set_ylabel('Total Infected People [%]')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.savefig('results/compensation_sidarthe_fpsp_{}_{}_{}.eps'.format(steps_high, steps_low, n_steps), dpi=1200)
plt.savefig('results/compensation_sidarthe_fpsp_{}_{}_{}.png'.format(steps_high, steps_low, n_steps), dpi=300)

## Supervisory Outer Loop

In [ ]:
### SIDARTHE Compensatory Behavior with Supervisory Outer Loop

# outer supervisory control
alpha_x = 0.4 # hystheresis for increasing duty cycle
alpha_y = 0. # hystheresis for decreasing duty cycle
x_init = 0
x_max = 14
period = x_max

step=0.2
compensation = np.array([step*i for i in range(0, 20)])
batch_size = compensation.shape[0]# number of different values for d

model = clds.agents.BatchSIDARTHE(s0=s0, 
                    alpha='a', # variable
                    beta='b', # variable
                    gamma='g', # variable
                    delta='d', #variable
                    epsilon=epsilon,
                    theta=theta,
                    zeta=zeta,
                    eta=eta,
                    mu=mu,
                    nu=nu,
                    tau=tau,
                    h=h,
                    rho=rho,
                    kappa=kappa,
                    xi=xi,
                    sigma=sigma,
                    N=N, 
                    batch_size=batch_size,
                    step_size=step_size)

# outer loop feedback: A+R+D
u = clds.Lambda(reset_fn= lambda: 0, step_fn= lambda x: x['model'][:,2] + x['model'][:,4]) # batch, channel

outer_loop = clds.agents.BatchOuterLoopFPSP(start=switching_start,
                            o='o', 
                            period=period, 
                            x_init=x_init,
                            x_max=x_max, 
                            alpha_x=alpha_x, 
                            alpha_y=alpha_y, 
                           batch_size=batch_size)

fpsp = clds.agents.BatchFPSP(beta_high=1,
                             beta_low=lockdown_effectiveness, 
                             steps_high='x', 
                             steps_low='y', 
                             suppression_start=suppression_start, 
                             switching_start=switching_start, 
                             batch_size=batch_size)

env = clds.Composite(order='sequential')
env.add(model, 
        pre=lambda x: {'a': x['fpsp']*alpha,
                       'b': x['fpsp']*beta,
                       'g': x['fpsp']*gamma,
                       'd': x['fpsp']*delta},
        out='model')
env.add(u, out='o')
env.add(outer_loop, out='ol')
env.add(fpsp, out='fpsp', pre= lambda x: {'x': x['ol'][:,0], 'y': x['ol'][:,1]})

o = [env.reset()]
for i in range(n_steps):
    if i > suppression_start:
        fpsp.beta_high = (1+compensation)
    o.append(env.step()[0])

s = np.array([x['model'] for x in o]).swapaxes(0, 1)

fig, ax = plt.subplots(figsize=(10, 8))

labels = ['S', 'I', 'D', 'A', 'R', 'T', 'H', 'E']
indices = [0, 4, 6, 8, 10, 11]
for i in indices:
    plt.plot(np.sum(s[i,::,1:6], axis=1)/N*100, label='d={:.2f}, T={}'.format(compensation[i], x_max))
ylim = plt.ylim()
plt.plot([suppression_start, suppression_start], ylim, '--', c='red')
plt.plot([switching_start, switching_start], ylim, '--', c='blue')
ax.set_xlabel('time (days)')
ax.set_ylabel('Total Infected People [%]')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.savefig('results/compensation_sidarthe_fpsp_T_{}_{}.png'.format(x_max, n_steps, step), dpi=300)
plt.savefig('results/compensation_sidarthe_fpsp_T_{}_{}.eps'.format(x_max, n_steps, step), dpi=1200)